In [2]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

# extracticing data for 2020

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [5]:
source = urllib.request.urlopen(url).read()
soup = bs.BeautifulSoup(source,'lxml')

In [6]:
tables = soup.find_all('table',class_='wikitable sortable')

In [11]:
len(tables)

4

In [13]:
type(tables[0])

bs4.element.Tag

In [14]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [16]:
df = pd.concat([df1,df2,df3,df4],axis=0)

In [17]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]
4,JANUARY,10,The Sonata,Screen Media Films,Andrew Desmond (director/screenplay); Arthur M...,[6]
...,...,...,...,...,...,...
33,DECEMBER,18,Coming 2 America,Paramount Pictures,"Craig Brewer (director); Kenya Barris, Barry W...",[171]
34,DECEMBER,18,The Father,Sony Pictures Classics / Film4,Florian Zeller (director/screenplay); Christop...,NaN
35,DECEMBER,23,The Croods: A New Age,Universal Pictures / DreamWorks Animation,"Joel Crawford (director); Kevin Hageman, Dan H...",[172]
36,DECEMBER,25,Wonder Woman 1984,Warner Bros. Pictures / DC Films / The Stone Q...,Patty Jenkins (director/screenplay); Geoff Joh...,[173]


In [18]:
df_2020 = df[['Title','Cast and crew']]

In [19]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...
...,...,...
33,Coming 2 America,"Craig Brewer (director); Kenya Barris, Barry W..."
34,The Father,Florian Zeller (director/screenplay); Christop...
35,The Croods: A New Age,"Joel Crawford (director); Kevin Hageman, Dan H..."
36,Wonder Woman 1984,Patty Jenkins (director/screenplay); Geoff Joh...


In [23]:
import json
file = open("./key.json",'r')
json_file = json.load(file)
api_key = json_file['api_key']

In [24]:
from tmdbv3api import TMDb
import requests
tmdb = TMDb()
tmdb.api_key = api_key

In [25]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres=[]
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN


In [26]:
df_2020['genres'] = df_2020['Title'].apply(get_genre)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
df_2020 = df_2020[['Title','Cast and crew','genres']]

In [28]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [29]:
df_2020['director_name'] = df_2020['Cast and crew'].apply(get_director)

In [30]:
def get_actor1(x):
    return(x.split('screenplay); ')[-1].split(', ')[0])

In [31]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].apply(get_actor1)

In [32]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [33]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].apply(get_actor2)

In [34]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [35]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].apply(get_actor3)

In [37]:
df_2020

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer
...,...,...,...,...,...,...,...
33,Coming 2 America,"Craig Brewer (director); Kenya Barris, Barry W...",Comedy,Craig Brewer,Eddie Murphy,Jermaine Fowler,Arsenio Hall
34,The Father,Florian Zeller (director/screenplay); Christop...,Drama,Florian Zeller,Anthony Hopkins,Olivia Colman,Mark Gatiss
35,The Croods: A New Age,"Joel Crawford (director); Kevin Hageman, Dan H...",Animation Adventure Family,Joel Crawford,Nicolas Cage,Emma Stone,Ryan Reynolds
36,Wonder Woman 1984,Patty Jenkins (director/screenplay); Geoff Joh...,Fantasy Action Adventure,Patty Jenkins,Gal Gadot,Chris Pine,Kristen Wiig


In [38]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [39]:
df_2020.drop(['Cast and crew'],axis=1)

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Horror Mystery,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,Action Horror Science Fiction Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Inherit the Viper,Drama Thriller Crime,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
4,The Sonata,Horror Thriller Mystery,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer
...,...,...,...,...,...,...
33,Coming 2 America,Comedy,Craig Brewer,Eddie Murphy,Jermaine Fowler,Arsenio Hall
34,The Father,Drama,Florian Zeller,Anthony Hopkins,Olivia Colman,Mark Gatiss
35,The Croods: A New Age,Animation Adventure Family,Joel Crawford,Nicolas Cage,Emma Stone,Ryan Reynolds
36,Wonder Woman 1984,Fantasy Action Adventure,Patty Jenkins,Gal Gadot,Chris Pine,Kristen Wiig


In [40]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [42]:
new_df20.isnull().sum()

director_name     0
actor_1_name      0
actor_2_name      2
actor_3_name     13
genres            0
movie_title       0
dtype: int64

In [43]:
new_df20['actor_2_name'] = new_df20['actor_2_name'].replace(np.nan, 'unknown')
new_df20['actor_3_name'] = new_df20['actor_3_name'].replace(np.nan, 'unknown')

In [44]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [45]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery,The Grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,Underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Drama Thriller Crime,Inherit the Viper,Josh Hartnett Margarita Levieva Chandler Riggs...
4,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer,Horror Thriller Mystery,The Sonata,Freya Tingley Simon Abkarian Rutger Hauer Andr...
...,...,...,...,...,...,...,...
33,Craig Brewer,Eddie Murphy,Jermaine Fowler,Arsenio Hall,Comedy,Coming 2 America,Eddie Murphy Jermaine Fowler Arsenio Hall Crai...
34,Florian Zeller,Anthony Hopkins,Olivia Colman,Mark Gatiss,Drama,The Father,Anthony Hopkins Olivia Colman Mark Gatiss Flor...
35,Joel Crawford,Nicolas Cage,Emma Stone,Ryan Reynolds,Animation Adventure Family,The Croods: A New Age,Nicolas Cage Emma Stone Ryan Reynolds Joel Cra...
36,Patty Jenkins,Gal Gadot,Chris Pine,Kristen Wiig,Fantasy Action Adventure,Wonder Woman 1984,Gal Gadot Chris Pine Kristen Wiig Patty Jenkin...


In [46]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [47]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

In [48]:
old_df = pd.read_csv('./clean_dataset/final_data.csv')

In [49]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5864,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5865,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action History,1917,George MacKay Dean-Charles Chapman Mark Strong...
5866,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...
5867,Chinonye Chukwu,Alfre Woodard,Wendell Pierce,Aldis Hodge,Drama,clemency,Alfre Woodard Wendell Pierce Aldis Hodge Chino...


In [50]:
final_df = pd.concat([old_df,new_df20],axis=0)

In [51]:
final_df.to_csv('./clean_dataset/main_data.csv',index=False)